# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [3]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [95]:
data = pd.read_csv('../data/customer_product_sales.csv')


In [8]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [22]:
total_qty = data.groupby(['CustomerID','ProductName']).sum('Quantity')
total_qty


SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [34]:
# total_qty = total_qty.fillna(0)
table=pd.pivot_table(total_qty, values='Quantity',  index=['ProductName'],columns='CustomerID' ,fill_value=0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [32]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [92]:

def get_similar_customers(customer_id):
    return distances[customer_id].sort_values(ascending=False)[1:].index   
get_similar_customers(33)[1:6]

Int64Index([3535, 3317, 2503, 3305, 1072], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [103]:
similars = list(get_similar_customers(33))[1:6]
def get_similars_products(similars):
    return total_qty.loc[similars]

similars_products = get_similars_products(similars)
similars_products

SalesID  ProductID  Quantity
CustomerID ProductName                                                  
3535       Anchovy Paste - 56 G Tube        3904179        131         1
           Apricots - Dried                 4143868        324         1
           Bacardi Breezer - Tropical       2311303         49         1
           Bandage - Fexible 1x3            5361156         93         1
           Beef - Chuck, Boneless           2496943        439         1
...                                             ...        ...       ...
1072       Wine - Chardonnay South          2255214        290         2
           Wine - Gato Negro Cabernet       5217371        315         1
           Wine - Red, Harrow Estates, Cab  2188105        365         1
           Wine - White, Colubia Cresh      2284191         44         1
           Yeast Dry - Fermipan             4213761         31         1

[256 rows x 3 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [108]:
# aggregated_products = similars_products.groupby('ProductName').sum('Quantity').sort_values(by="Quantity", ascending=False)
def get_aggregated_products(similars_products):
    return similars_products.groupby('ProductName').sum('Quantity').sort_values(by="Quantity", ascending=False)

aggregated_products = get_aggregated_products(similars_products)

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [110]:

merged_data = aggregated_products.merge(table[33].reset_index(), left_on='ProductName', right_on='ProductName')
list(merged_data.loc[merged_data[33] == 0][:5]['ProductName'])


['Knife Plastic - White',
 'Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Wine - Blue Nun Qualitatswein',
 'Beef Wellington']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [115]:
recommendations = []
customer_ids = data['CustomerID'].unique()
r = pd.DataFrame()
for c_id in customer_ids:
    similars = list(get_similar_customers(c_id))[1:6]
    similars_products = get_similars_products(similars)
    aggregated_products = get_aggregated_products(similars_products)
    merged_data = aggregated_products.merge(table[c_id].reset_index(), left_on='ProductName', right_on='ProductName')
    recommendations+=list(merged_data.loc[merged_data[c_id] == 0][:5]['ProductName'])
    r[c_id]=pd.Series(recommendations)
recommendations


['Lime Cordial - Roses',
 'Chicken - Soup Base',
 'Dc Hikiage Hira Huba',
 'Jagermeister',
 'Wine - Prosecco Valdobiaddene',
 'Beef - Rib Eye Aaa',
 'Chicken - Soup Base',
 'Pepsi - Diet, 355 Ml',
 'Wine - Vineland Estate Semi - Dry',
 'Sun - Dried Tomatoes',
 'Oregano - Dry, Rubbed',
 'Chestnuts - Whole,canned',
 'Puree - Mocha',
 'Bread - Italian Corn Meal Poly',
 'Smirnoff Green Apple Twist',
 'Pernod',
 'Flavouring - Orange',
 'Squid - Tubes / Tenticles 10/20',
 'Wine - Magnotta, Merlot Sr Vqa',
 'Milk - 2%',
 'Vanilla Beans',
 'Lettuce - Frisee',
 'Yogurt - Blueberry, 175 Gr',
 'Pop Shoppe Cream Soda',
 'Beef - Inside Round',
 'Cocoa Butter',
 'Turkey - Oven Roast Breast',
 'Fond - Neutral',
 'Scallop - St. Jaques',
 'Ecolab - Lime - A - Way 4/4 L',
 'Pomello',
 'Sole - Dover, Whole, Fresh',
 'Juice - Lime',
 'Soup - Campbells Tomato Ravioli',
 'Potatoes - Instant, Mashed',
 'Cheese - Brie, Triple Creme',
 'Cheese - Mozzarella',
 'Bread - French Baquette',
 'Bandage - Flexible Neo

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [116]:
r

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,...,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses,Lime Cordial - Roses
1,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,...,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base,Chicken - Soup Base
2,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,...,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba,Dc Hikiage Hira Huba
3,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,...,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister,Jagermeister
4,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,...,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene,Wine - Prosecco Valdobiaddene


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [117]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'jaccard'))), 
                         index=table.columns, columns=table.columns)

recommendations = []
customer_ids = data['CustomerID'].unique()
r = pd.DataFrame()
for c_id in customer_ids:
    similars = list(get_similar_customers(c_id))[1:6]
    similars_products = get_similars_products(similars)
    aggregated_products = get_aggregated_products(similars_products)
    merged_data = aggregated_products.merge(table[c_id].reset_index(), left_on='ProductName', right_on='ProductName')
    recommendations+=list(merged_data.loc[merged_data[c_id] == 0][:5]['ProductName'])
    r[c_id]=pd.Series(recommendations)
r

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,"Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme",...,"Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme","Cheese - Brie, Triple Creme"
1,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,...,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend,V8 - Berry Blend
2,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,...,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L,Ecolab - Mikroklene 4/4 L
3,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,...,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange,Flavouring - Orange
4,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,...,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop,Tomatoes Tear Drop
